In [ ]:
import torch
from lib.utils import load_raw_list,load_psd,load_raw
from torch.nn.functional import one_hot
import pandas as pd
import numpy as np
from tqdm import tqdm
idx = 0
i = 0
zeros = []
y_final = torch.Tensor()

In [ ]:
## Run this cell with range from 0 to 10, 10 to 20, 20 to 30, 30 to 32
ret = pd.DataFrame()
idxs = list(range(15,32))
raw = load_raw(idxs[0])
df = load_psd(idxs[0])
eeg = raw.get_data(picks='EEG')[0]
X = pd.DataFrame(eeg.reshape(-1,5000))
y = df['label']

append = pd.concat([y,X],axis=1)
ret = pd.concat([ret,append])
for i in idxs[1:]:
    raw = load_raw(i)
    df = load_psd(i)
    eeg = raw.get_data(picks='EEG')[0]
    X = pd.DataFrame(eeg.reshape(-1,5000))
    y = df['label']

    append = pd.concat([y,X],axis=1)
    ret = pd.concat([ret,append])

ret = ret.reset_index(drop=True)
ret = ret[ret['label'] != 'X']

y = np.array(pd.Categorical(ret.pop('label')).codes)
X = ret.to_numpy()

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).reshape(-1,1).long()

In [ ]:
for sample,label in tqdm(zip(X,y)):
    sample = sample-sample.mean()
    if(sample.std().item()==0):
        zeros.append(i)
        i+=1
        continue
    sample = sample/sample.std()
    torch.save(sample.clone(),f'data_pt/{idx}.pt')
    y_final = torch.cat([y_final,label])
    i+=1
    idx += 1

In [ ]:
y = one_hot(y_final.reshape(-1,1).long(),num_classes=3).reshape(-1,3).float()

In [ ]:
torch.save(y,'data_pt/labels.pt')

In [ ]:
len(y)==idx

In [ ]:
idx

In [ ]:
from sklearn.model_selection import train_test_split
import torch
y = torch.load('data_pt/labels.pt')
train_idx,test_idx,_,_ = train_test_split(range(len(y)),y,shuffle=True,test_size=.2,random_state=0)

In [ ]:
os.system('mkdir data_pt/train')
os.system('mkdir data_pt/test')

for i,idx in enumerate(train_idx):
    os.system(f'mv data_pt/{idx}.pt data_pt/train/{i}.pt')
for i,idx in enumerate(test_idx):
    os.system(f'mv data_pt/{idx}.pt data_pt/test/{i}.pt')
torch.save(y[train_idx],'data_pt/y_train.pt')
torch.save(y[test_idx],'data_pt/y_test.pt')

In [ ]:
# random oversampling
import pandas as pd
y_train_df = pd.DataFrame(torch.load(f'data/pt_bal/y_train.pt').argmax(axis=1).numpy())

In [ ]:
from numpy.random import randint
y_train_df.value_counts()

In [ ]:
import numpy as np
nmajority = y_train_df.value_counts()[2]
sidx = np.array(y_train_df[y_train_df[0] == 1].index)
pidx = np.array(y_train_df[y_train_df[0] == 0].index)

In [ ]:
n_s = len(sidx)
n_p = len(pidx)
n_train = len(y_train_df)

In [ ]:
n_train

In [ ]:
cur_idx = n_train

In [ ]:
y_train = list(y_train_df.to_numpy().flatten())

In [ ]:
for i in range(nmajority-n_p):
    rand_idx = np.random.choice(pidx)
    os.system(f'cp data/pt_bal/train/{rand_idx}.pt data/pt_bal/train/{cur_idx}.pt')
    cur_idx += 1
    y_train.append(0)

In [ ]:
for i in range(nmajority-n_s):
    rand_idx = np.random.choice(sidx)
    os.system(f'cp data/pt_bal/train/{rand_idx}.pt data/pt_bal/train/{cur_idx}.pt')
    cur_idx += 1
    y_train.append(1)

In [ ]:
y = np.array(pd.Categorical(y_train).codes)
y = torch.from_numpy(y).reshape(-1,1).long()
from torch.nn.functional import one_hot
y = one_hot(y,num_classes=3).reshape(-1,3).float()
torch.save(y,'data/pt_bal/y_train.pt')

In [ ]:
# norm
dir = 'pt'
import torch
import matplotlib.pyplot as plt
stds = []
for file in os.listdir(f'data/{dir}/train'):
    X = torch.load(f'data/{dir}/train/{file}')
    X_centered = X-X.mean()
    if(X_centered.std().item() == 0):
        print("0 std",file)
        stds.append(file)
    # X_streched = X/X_centered.std()

In [ ]:
for file in stds:
    plt.plot(torch.load(f'data/{dir}/train/{file}'))

In [1]:
from lib.utils import load_raw_list,load_psd,load_raw
X,y = load_raw_list([0])

/home/andrew/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch

In [7]:
X = torch.cat([X[:-2],X[1:-1],X[2:]],axis=1)
y = y[1:-1]

In [10]:
y.shape

torch.Size([8638, 3])

In [ ]:
import torch

# i = 0
# for fileindex in tqdm(range(32)):
#     X,y = load_raw_list([fileindex])
#     X = torch.cat([X[:-2],X[1:-1],X[2:]],axis=1)
#     y = y[1:-1]
#     for (Xi,yi) in zip(X,y):
#         torch.save((Xi.clone(),yi.clone()),f'windowed/{i}.pt')
#         i += 1